# 1. Set Up the Notebook

In [1]:
# test_rag_pipeline.ipynb
# Import necessary modules
from rag_skeleton.data_processing import DataProcessor
from rag_skeleton.retrieval import DocumentRetriever
from rag_skeleton.rag import RAGPipeline

# Initialize paths and model names
data_path = "data/raw"  # Path where your PDFs are stored
vectordb_path = "vectordb"  # Path for vector database
embedding_model_name = "Alibaba-NLP/gte-large-en-v1.5"
model_name = "meta-llama/Llama-3.2-3B-Instruct"

c:\Users\hsayeed\miniconda3\envs\rag_skeleton_seperate_project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Test Loading PDFs

In [2]:
# Step 1: Load PDFs and inspect content
processor = DataProcessor(data_path=data_path)
documents = processor.load_documents(enrich_metadata=True)  # Enable metadata enrichment

print("Loaded Documents:")
for i, doc in enumerate(documents[:3]):  # Display the first 3 documents for brevity
    print(f"Document {i+1}:")
    print("Content:", doc.page_content[:500])  # Show the first 500 characters of the content
    print("Metadata:", doc.metadata)
    print("\n---\n")

Loaded 16 documents.
Loaded Documents:
Document 1:
Content: A Comprehensive Overview of Materials Science 
1. Introduction to Materials Science 
Materials science is an interdisciplinary field that focuses on understanding and manipulating the 
properties, composition, and processing of materials to meet the needs of modern technology and 
innovation. It draws on principles from chemistry, physics, and engineering to explain how the 
atomic structure, microstructure, and composition of materials affect their macroscopic 
properties. From everyday items l
Metadata: {'source': 'data/raw\\A Comprehensive Overview of Materials Science.pdf', 'file_path': 'data/raw\\A Comprehensive Overview of Materials Science.pdf', 'page': 0, 'total_pages': 5, 'format': 'PDF 1.7', 'title': '', 'author': 'Hasan Sayeed', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word LTSC', 'producer': 'Microsoft® Word LTSC', 'creationDate': "D:20241110201707-07'00'", 'modDate': "D:20241110201707-07'00'", 'trappe

# 3. Test Retriever for Context Retrieval

In [3]:
# Step 2: Retrieve context for a sample query
retriever = DocumentRetriever(vectordb_path=vectordb_path, embedding_model_name=embedding_model_name)
# retriever.index_documents(documents)  # Index the loaded documents

# Define a sample question
question = "Can you explain the environmental impact of traditional materials, especially in terms of energy consumption and pollution?"

# Retrieve relevant documents
retrieved_docs = retriever.get_retriever().invoke(question)

print("Retrieved Context:")
for i, doc in enumerate(retrieved_docs[:3]):  # Display the first 3 retrieved documents
    print(f"Retrieved Document {i+1}:")
    print("Content:", doc.page_content[:500])  # Show first 500 characters of the retrieved content
    print("Metadata:", doc.metadata)
    print("\n---\n")

Retrieved Context:
Retrieved Document 1:
Content: Environmental Impact and Sustainable 
Materials in Materials Science 
1. Introduction to Environmental Impact and Sustainable Materials 
The global demand for materials has grown significantly due to industrialization, population 
growth, and technological advancements. While materials are essential to modern society, their 
production and use often come with environmental costs, including resource depletion, energy 
consumption, pollution, and waste generation. Sustainable materials science see
Metadata: {'author': 'Hasan Sayeed', 'creationDate': "D:20241110202300-07'00'", 'creator': 'Microsoft® Word LTSC', 'file_path': 'data/raw\\Environmental Impact and Sustainable Materials in Materials Science.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20241110202300-07'00'", 'page': 0, 'producer': 'Microsoft® Word LTSC', 'source': 'data/raw\\Environmental Impact and Sustainable Materials in Materials Science.pdf', 'subject': '', 'tit

# 4. Test Prompt Generation with RAGPipeline

In [4]:
# Step 3: Test prompt generation with RAGPipeline
rag_pipeline = RAGPipeline(vectordb_path=vectordb_path, embedding_model_name=embedding_model_name, model_name=model_name)
rag_pipeline.setup_pipeline()  # Initialize and set up the pipeline

# Define the question, context, and initialize history for prompt formatting
history = ""  # Start with an empty history or use some predefined history if available
formatted_prompt = rag_pipeline.prompt.format(history=history, context=retrieved_docs[0].page_content, question=question)

print("Generated Prompt:")
print(formatted_prompt)

Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.54s/it]


RAG Pipeline is ready for queries.
Generated Prompt:

        <|start_header_id|>system<|end_header_id|>
        You are a knowledgeable assistant specializing in materials science. Answer each question directly and concisely, providing only the necessary information in a straightforward manner, using only the provided context and conversation history. Do NOT reference the question or instructions in the prompt.
        If you lack sufficient information, respond with "I do not know". Don't fabricate answers.
        <|eot_id|>
        <|start_header_id|>user<|end_header_id|>
        Question: Can you explain the environmental impact of traditional materials, especially in terms of energy consumption and pollution?
        Context: Environmental Impact and Sustainable 
Materials in Materials Science 
1. Introduction to Environmental Impact and Sustainable Materials 
The global demand for materials has grown significantly due to industrialization, population 
growth, and technological a

# 5. Full Pipeline Test: Get Response

In [5]:
# Step 4: Full pipeline test - Generate a response
response = rag_pipeline.get_response(question)

print("Generated Response:")
print(response)

Generated Response:
 The environmental impact of traditional materials is significant, resulting in resource depletion, energy consumption, pollution, and waste generation. Key impacts include:

Resource Depletion: Extraction of non-renewable resources leads to depletion of Earth's reserves.

Energy Consumption: Material production is energy-intensive, often derived from fossil fuels, increasing greenhouse gas emissions.

Pollution and Emissions: Processing materials releases pollutants into the air, water, and soil, contributing to air pollution.

Waste Generation: Industrial processes generate large amounts of waste, including hazardous waste from outdated devices and long-lasting pollution from conventional plastics.

These traditional materials have led to a growing emphasis on sustainable alternatives that aim to mitigate resource depletion, reduce emissions, and minimize waste.

For further reference, look at:
- data/raw\Environmental Impact and Sustainable Materials in Materials

# Test multiple questions and prompt

In [ ]:
from rag_skeleton.rag import RAGPipeline

# Initialize the RAG pipeline
vectordb_path = "vectordb"  # Replace with your vector database path
model_name = "meta-llama/Llama-3.2-3B-Instruct"  # Replace with your model name
rag_pipeline = RAGPipeline(vectordb_path=vectordb_path, model_name=model_name)
rag_pipeline.setup_pipeline()

# Define the questions you want to test with history
questions = [
    "Can you explain the environmental impact of traditional materials, especially in terms of energy consumption and pollution?",
    "Given the environmental concerns with traditional materials, what sustainable alternatives are commonly used to reduce this impact, and how do they compare in terms of recyclability and biodegradability?",
    "Among the sustainable materials you mentioned, how does green concrete specifically contribute to reducing emissions, and what are some of its unique properties compared to traditional concrete?",
    "How does the use of green concrete and other sustainable materials align with the principles of the circular economy, and what role does recycling play in this context?"
]

# Preview the prompt that will be passed to the LLM for each question
for i, question in enumerate(questions):
    print(f"Preview of Prompt for Question {i+1}:")
    prompt_text = rag_pipeline.preview_prompt(question)
    print(prompt_text)
    print("\n" + "="*80 + "\n")

    # Optionally, invoke the response to update the history
    response = rag_pipeline.get_response(question)
